In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [3]:
batch_size= 32
input_size = (64, 64)
optimizer = 'adam'

In [4]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [26]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [27]:
import keras

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np


image_shape = (64,64,3)
np.random.seed(1000)
model = Sequential()
#First Convolutional layer
model.add(Conv2D(filters=96, input_shape=image_shape, kernel_size=(11,11), strides=(4,4),padding='valid'))
model.add(Activation('relu'))

#Max Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

#Second Convolutional layer
model.add(Conv2D(filters=96, kernel_size=(5,5), strides=(1,1), padding='same'))
model.add(Activation('relu'))

#Max Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

#Third Convolutional layer
model.add(Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))

#Fourth Convolutional layer
model.add(Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))

#Fifth Convolutional layer
model.add(Conv2D(filters=22, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))

#Max Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))

#Passing it to a fully connected layer, here we do the flatten!
model.add(Flatten())

#First Fully Connected layer has 4096 neurons
model.add(Dense(300, input_shape=(64*64*3,)))
model.add(Activation('relu'))

#Add dropout to prevent overfitting
model.add(Dropout(0.5))

#Second Fully Connected layer
model.add(Dense(200))
model.add(Activation('relu'))

#Add Dropout
model.add(Dropout(0.5))

#Output layer
model.add(Dense(27))
model.add(Activation('softmax'))

model.summary()

model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=["accuracy", recall_m, precision_m, f1_m])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 14, 14, 96)        34944     
_________________________________________________________________
activation_8 (Activation)    (None, 14, 14, 96)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 96)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 6, 6, 96)          230496    
_________________________________________________________________
activation_9 (Activation)    (None, 6, 6, 96)          0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 2, 2, 96)          0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 2, 2, 16)         

In [28]:
path_train_data ="char_recog/train"
path_test_data ="char_recog/val"

In [8]:
import splitfolders

splitfolders.ratio('char_training', output="char_recog", seed=1337, ratio=(0.8, 0.2))

Copying files: 2500 files [00:11, 175.23 files/s]

KeyboardInterrupt: 

In [ ]:
subfolder =["train", "val"]
for sf in subfolder:
    mainfolder= f"char_recog/{sf}"
    folders = os.listdir(mainfolder)
    for folder in  folders:
        files = os.listdir(f"{mainfolder}/{folder}")
        for f in files:
            impath = f"{mainfolder}/{folder}/{f}"
            newfile = "".join(f.split(".")[0:-1])
            newpath = f"{mainfolder}/{folder}/{newfile}.png"
            img = Image.open(impath).convert("RGBA")
            img.save(newpath)

In [16]:
gen_train_data = ImageDataGenerator(rescale=1./255, width_shift_range=0.1,
        shear_range=0.15, zoom_range=0.1,
        channel_shift_range=10., horizontal_flip=False)

In [17]:
gen_test_data = ImageDataGenerator(rescale=1./255)

In [23]:
train_generator = gen_train_data.flow_from_directory(
                path_train_data, 
                target_size=input_size, 
                batch_size=batch_size,
                class_mode='categorical')

Found 4425 images belonging to 27 classes.


In [24]:
val_generator = gen_train_data.flow_from_directory(
                path_test_data,  # this is the target directory
                target_size=input_size,  # all images will be resized to 64x64
                batch_size=batch_size,
                class_mode='categorical',subset='training')

Found 1112 images belonging to 27 classes.


In [29]:
weights_pretrain = model.fit_generator(
        train_generator,
        steps_per_epoch= train_generator.samples // batch_size,
        epochs=100,
        validation_data=val_generator,
        validation_steps=val_generator.samples // batch_size)
model.save_weights(f'data+.h5')

Epoch 1/100
138/138 [==============================] - 14s 79ms/step - loss: 3.1651 - accuracy: 0.0633 - recall_m: 0.0011 - precision_m: 0.0166 - f1_m: 0.0019 - val_loss: 1.8182 - val_accuracy: 0.3695 - val_recall_m: 0.1287 - val_precision_m: 0.6680 - val_f1_m: 0.2132
Epoch 2/100
138/138 [==============================] - 10s 73ms/step - loss: 1.6982 - accuracy: 0.4270 - recall_m: 0.2362 - precision_m: 0.6018 - f1_m: 0.3252 - val_loss: 0.6840 - val_accuracy: 0.8061 - val_recall_m: 0.7307 - val_precision_m: 0.9107 - val_f1_m: 0.8096
Epoch 3/100
138/138 [==============================] - 10s 72ms/step - loss: 0.8792 - accuracy: 0.7331 - recall_m: 0.6441 - precision_m: 0.8498 - f1_m: 0.7298 - val_loss: 0.5683 - val_accuracy: 0.8263 - val_recall_m: 0.7454 - val_precision_m: 0.9297 - val_f1_m: 0.8260
Epoch 4/100
138/138 [==============================] - 10s 74ms/step - loss: 0.5880 - accuracy: 0.8296 - recall_m: 0.7616 - precision_m: 0.9006 - f1_m: 0.8236 - val_loss: 0.3845 - val_accuracy: